Connect to trt table in mysql database and save TRT locations in csv file

In [15]:
import pyodbc
import pandas as pd

In [16]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=EDWPRDRPTDB;'
                      'Database=EDW;'
                      'Trusted_Connection=yes;')    

In [21]:
df = pd.read_sql_query('SELECT * from EDW.dimTRTstore', conn)

c:\Users\fashaikh\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [22]:
df.drop_duplicates(subset=['Latitude', 'Longitude'], inplace=True)

In [23]:
# Select open TRT in the US that are not Charging or Parking locations

df = df[df['Status'] == 'Open']
df = df[df['Country'] == 'US']
#df = df[df['Latitude'] != 0]
#df = df[df['Latitude'].notna()]

In [24]:
df['Longitude'].value_counts(dropna=False)

-99.901813     2
-119.353754    2
-74.006017     2
-111.093731    2
-122.057821    2
              ..
-123.187499    1
-81.934763     1
-112.103895    1
-70.930377     1
-117.097588    1
Name: Longitude, Length: 8261, dtype: int64

In [25]:

df = df[df['Types_360'] != 'Parking']

In [26]:
df = df[df['Types_360'] != 'Charging']

In [27]:
len(df)

2686

In [28]:
df['Types_360'].value_counts()

Vendor                                      1120
Support                                      259
Facilities                                   215
Charging,Tesla Center                        211
Tesla Center                                 189
Facilities,Support                            10
Parking,Tesla Center                           7
Support,Tesla Center                           7
Charging,Support,Tesla Center                  7
Charging,Facilities,Tesla Center               5
Facilities,Tesla Center                        5
Parking,Support                                5
Facilities,Parking                             5
Charging,Facilities                            2
Facilities,Support,Tesla Center                2
Tesla Center,Vendor                            2
Charging,Facilities,Support                    1
Charging,Parking,Tesla Center                  1
Charging,Facilities,Support,Tesla Center       1
Charging,Support                               1
Parking,Vendor      

In [29]:
dtrt = df.loc[:, ['TRT_ID', 'Latitude', 'Longitude', 'System_Name']]

In [ ]:
dtrt.to_parquet("dimTRTstore.parquet", index=False)

In [30]:
dtrt.to_csv("dimTRTstore.csv", index=False)

In [31]:
conn.close()